In [27]:
type Kind = 
| Start
| End
| Step
type Coord = {
    line: int
    col: int
    height: int
    kind: Kind
}
module Coord =
    let value line col (character:char) =
        let toInt c = (c |> int) - 96
        let aPos = { line = line; col = col; height = -1; kind = Step }
        match character with
        | 'S' -> { aPos with height = 'a' |> toInt; kind = Start }
        | 'E' -> { aPos with height = 'z' |> toInt; kind = End }
        | _ -> { aPos with height = character |> toInt }
    let mapIt values =
        values
        |> Seq.mapi (fun idx line -> 
            line 
            |> Seq.mapi (fun idy v -> v |> value idx idy )
            |> List.ofSeq)
        |> List.ofSeq

    let start map =
        map
        |> Seq.concat
        |> Seq.filter (fun it -> match it.kind with | Kind.Start _ -> true | _ -> false)
        |> Seq.head
    
    let goal map =
        map
        |> Seq.concat
        |> Seq.filter (fun it -> match it.kind with | Kind.End _ -> true | _ -> false)
        |> Seq.head

type AStar = {
    goal: Coord
    openSet: Set<Coord>
    cameFrom: Map<Coord, Coord>
    gScore: Map<Coord, int>
    fScore: Map<Coord, int>
}
module AStar =
    let score (pos:Coord) (goal:Coord) =
        let colOffset = Math.Abs(pos.col - goal.col)
        let lineOffset = Math.Abs(pos.line - goal.line)
        colOffset + lineOffset

    let neighbours (map: Coord list list) (pos:Coord) =
        let  height = ( map |> List.length ) - 1
        let width = ( map.[0] |> List.length ) - 1

        seq {
            if pos.line < height then 
                yield map.[pos.line + 1].[pos.col]
            
            if pos.line > 0 then 
                yield map.[pos.line - 1].[pos.col]
            
            if pos.col < width then 
                yield map.[pos.line].[pos.col + 1]
            
            if pos.col > 0 then 
                yield map.[pos.line].[pos.col - 1]
        }
        |> Seq.filter (fun it -> it.height - pos.height <= 1 )

    let create start goal =
        {  
            goal = goal
            openSet = Set.singleton start
            cameFrom = Map.empty
            gScore = [(start, 0)] |> Map.ofList
            fScore = [ (start, ( goal |> score start ))] |> Map.ofList
        }

    let next (state:AStar) =
        state.openSet
        |> Seq.sortBy (fun it -> state.fScore.[it])
        |> Seq.head

    let getPath state =
        let rec gp acc current (state:AStar) =
            if state.cameFrom.ContainsKey(current) then
                let next = state.cameFrom.Item current
                gp (acc |> Seq.append [next]) next state
            else 
                acc
        gp Seq.empty (state.goal) state

    let rec find (state:AStar) (map:Coord list list) =
        if state.openSet.IsEmpty then 
            display "fail"
            state 
        else

        let current = state |> next

        if current.kind = Kind.End then 
            display "⭐"
            state 
        else 

        let openSet = state.openSet |> Set.remove current

        let folder (state:AStar) (neighbour:Coord) =
            let tentative = (state.gScore.Item current) + 1
            let actual = state.gScore.TryFind neighbour |> Option.defaultValue 999999
            if tentative < actual then
                let fScore = tentative + (state.goal |> score neighbour)
                { state with 
                    cameFrom = state.cameFrom.Add(neighbour, current)
                    gScore = state.gScore.Change(neighbour, Option.orElse (Some tentative))
                    fScore = state.fScore.Change(neighbour, Option.orElse (Some fScore))
                    openSet = openSet |> Set.add neighbour
                }
            else
                { state with openSet = openSet }

        let newState = 
            current
            |> neighbours map
            |> Seq.fold folder { state with openSet = openSet }

        find newState map
            
let ResolutionFolder = __SOURCE_DIRECTORY__
let lines = File.ReadLines( ResolutionFolder + "/testcase12.txt")

let map =
    lines
    |> Coord.mapIt

let start = map |> Coord.start

let goal = map |> Coord.goal

let solution =
    map |> AStar.find (AStar.create start goal)

solution

goal,openSet,cameFrom,gScore,fScore
Coord line: 2 col: 5 height: 26 kind: End,FSharpSet<Coord>,"FSharpMap<Coord,Coord> - Key: Coord line: 0 col: 1 height: 1 kind: Step Value: Coord line: 0 col: 0 height: 1 kind: Start - Key: Coord line: 0 col: 2 height: 2 kind: Step Value: Coord line: 0 col: 1 height: 1 kind: Step - Key: Coord line: 1 col: 0 height: 1 kind: Step Value: Coord line: 0 col: 0 height: 1 kind: Start - Key: Coord line: 1 col: 1 height: 2 kind: Step Value: Coord line: 0 col: 1 height: 1 kind: Step","FSharpMap<Coord,Int32> - Key: Coord line: 0 col: 0 height: 1 kind: Start Value: 0 - Key: Coord line: 0 col: 1 height: 1 kind: Step Value: 1 - Key: Coord line: 0 col: 2 height: 2 kind: Step Value: 2 - Key: Coord line: 1 col: 0 height: 1 kind: Step Value: 1 - Key: Coord line: 1 col: 1 height: 2 kind: Step Value: 2","FSharpMap<Coord,Int32> - Key: Coord line: 0 col: 0 height: 1 kind: Start Value: 7 - Key: Coord line: 0 col: 1 height: 1 kind: Step Value: 7 - Key: Coord line: 0 col: 2 height: 2 kind: Step Value: 7 - Key: Coord line: 1 col: 0 height: 1 kind: Step Value: 7 - Key: Coord line: 1 col: 1 height: 2 kind: Step Value: 7"
